In [3]:
# Bloque 1: Instalación de las librerías necesarias

# Si no tienes las librerías instaladas o necesitas actualizar, puedes usar lo siguiente:
#!pip install --upgrade numpy
#!pip install --upgrade pandas
#!pip install --upgrade nltk
#!pip install --upgrade openpyxl  
#!pip install --upgrade scikit-learn  


   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.0 MB 2.6 MB/s eta 0:00:04
   --- ------------------------------------ 1.0/11.0 MB 1.7 MB/s eta 0:00:06
   ---- ----------------------------------- 1.3/11.0 MB 1.8 MB/s eta 0:00:06
   ------ --------------------------------- 1.8/11.0 MB 1.7 MB/s eta 0:00:06
   -------- ------------------------------- 2.4/11.0 MB 1.8 MB/s eta 0:00:05
   --------- ------------------------------ 2.6/11.0 MB 1.8 MB/s eta 0:00:05
   ----------- ---------------------------- 3.1/11.0 MB 1.8 MB/s eta 0:00:05
   ------------ --------------------------- 3.4/11.0 MB 1.8 MB/s eta 0:00:05
   --------------- ------------------------ 4.2/11.0 MB 1.9 MB/s eta 0:00:04
   ------------------- -------------------- 5.2/11.0 MB 2.2 MB/s eta 0:00:03
   --------------------- ------------------ 6.0/11.0 MB 2.4 MB/s eta 0:00:03
   ----------

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blis 1.0.1 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [4]:
# Bloque 2: Importar librerías necesarias y cargar el dataset

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# Descargar recursos de NLTK necesarios (stopwords y tokenizer)
nltk.download('punkt')
nltk.download('stopwords')

# Cargar el archivo de datos (debes tener 'Corpus-Agro.xlsx' en tu directorio)
df = pd.read_excel('Corpus-Agro.xlsx')

# Ver las primeras filas del dataframe para asegurarnos de que se ha cargado correctamente
df.head()


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# Bloque 3: Limpiar los resúmenes del dataset utilizando un análisis léxico

# Definir el conjunto de stopwords en inglés
stop_words = set(stopwords.words('english'))

# Definir la función para limpiar los resúmenes (análisis léxico)
def limpiar_texto_lexico(texto):
    if isinstance(texto, float):  # Comprobar si el texto es un número o NaN
        return ""
    
    # Convertir el texto a minúsculas
    texto = texto.lower()
    
    # Tokenizar el texto (separar palabras)
    palabras = word_tokenize(texto)
    
    # Filtrar stopwords y puntuación
    palabras_limpias = [word for word in palabras if word not in stop_words and word not in string.punctuation]
    
    return " ".join(palabras_limpias)

# Aplicar la función de limpieza a todos los resúmenes
df['Resumen_Limpio'] = df['Resumen'].apply(limpiar_texto_lexico)

# Mostrar algunos resultados limpios
df[['Resumen', 'Resumen_Limpio']].head()


In [ ]:
# Bloque 4: Entrenar un modelo Word2Vec utilizando los resúmenes limpios

from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

# Tokenizar todos los resúmenes limpios para entrenar Word2Vec
resumenes_tokenizados = [word_tokenize(texto) for texto in df['Resumen_Limpio']]

# Entrenar el modelo Word2Vec
modelo_word2vec = Word2Vec(resumenes_tokenizados, vector_size=100, window=5, min_count=2, workers=4)

# Guardar el modelo entrenado (opcional, si quieres reutilizarlo más tarde)
modelo_word2vec.save("modelo_word2vec.model")


In [ ]:
# Bloque 5: Solicitar una consulta al usuario y procesarla

# Pedir al usuario que ingrese su consulta
consulta_usuario = input("Por favor, ingresa tu consulta sobre agricultura: ").lower()

# Tokenizar la consulta del usuario
tokens_consulta = word_tokenize(consulta_usuario)

# Generar un vector promedio para la consulta
vector_consulta = np.mean([modelo_word2vec.wv[token] for token in tokens_consulta if token in modelo_word2vec.wv], axis=0)

# Si no se encuentran palabras en el modelo, manejar el caso
if np.isnan(vector_consulta).any():
    print("La consulta no contiene palabras reconocidas por el modelo.")
else:
    print("Consulta vectorizada correctamente.")


In [ ]:
# Bloque 6: Calcular la similitud coseno entre la consulta y los resúmenes

# Vectorizar cada resumen utilizando el modelo Word2Vec (promediando los vectores de palabras)
def vectorizar_resumen(texto):
    tokens = word_tokenize(texto)
    vector = np.mean([modelo_word2vec.wv[token] for token in tokens if token in modelo_word2vec.wv], axis=0)
    return vector

# Aplicar la vectorización a todos los resúmenes limpios
df['Vector_Resumen'] = df['Resumen_Limpio'].apply(vectorizar_resumen)

# Calcular la similitud coseno entre la consulta y los resúmenes
df['Similitud'] = df['Vector_Resumen'].apply(lambda x: cosine_similarity([x], [vector_consulta])[0][0])

# Ordenar los resúmenes por similitud (de mayor a menor)
df_resultados = df[['URL del Documento', 'Resumen', 'Similitud']].sort_values(by='Similitud', ascending=False)

# Mostrar los primeros n resultados más relevantes
n = 5  # Puedes cambiar el número de resultados a mostrar
print(df_resultados.head(n))
